In [1]:
#"""this a jupyter notebook to produce an enhanced visualization output for the app"""
#import os
#from ase.io import read,write
#from ase.visualize import view
#os.chdir(os.getcwd())
#x=read('test.vasp')
#view(x,viewer='ngl')

In [ ]:
from ase.io import read
from ase.atoms import Atoms
from ase.visualize import view
import os
from tkinter import *
from ase.io import read,write
from ase.visualize import view
import nglview as nv
from time import sleep
from ipywidgets import interact
from IPython.display import display
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual,interactive_output
import ipywidgets as widgets
from tkinter import Tk,Text,OptionMenu,filedialog,END,Button,StringVar
from functools import partial
def view_structure(atoms, snapshot=-1, spacefill=True, show_cell=True):    
        picture = nv.show_ase(atoms)
        picture.add_representation('ball+stick', radiusType='data', radius='0.2', bondRadius='1.0', bondScale=1.0)
        picture.add_unitcell()
        return picture
def cartesian(a,b,c):
    aa=list(i for i in range(a))
    bb=list(i for i in range(b)) 
    cc=list(i for i in range(c)) 
    product=[]
    for i in range(len(aa)):
        for j in range(len(bb)):
            for k in range(len(cc)):
                product.append(str(aa[i])+str(bb[j])+str(cc[k]))
    del(product[0])
    return product
def distance(a,b):
    #np.sqrt((x.positions[10][0]-x.positions[20][0])**2+(x.positions[10][1]-x.positions[20][1])**2+(x.positions[10][2]-x.positions[20][2])**2)
    return np.sqrt((a[0]-b[0])**2+(a[1]-b[1])**2+(a[2]-b[2])**2)
def permut2(n):
    x=[]
    y=[]
    for i in range(n):
        x.append(i)
    for i in range(len(x)-1):
        y.append([(x[i],z) for z in x[i+1:]])
    z=[]
    for a in y:
        z=z+a
    return z
def str_permut2(s):
    x=[]
    y=[]
    for i in range(len(s)):
        x.append(s[i])
    for i in range(len(x)-1):
        y.append([(x[i],z) for z in x[i+1:]])
    z=[]
    for a in y:
        z=z+a
    mix=[]
    for a in z:
        mix.append(a[0]+'-'+a[1])
    return mix
os.chdir(os.getcwd())
#os.chdir("C:\\Users\\br\\Desktop\\Pychem\\PyQChem-3.0\\PyQChem 2.0\\QChem")
f=read('test.vasp')
l=[]
for i in range(len(f.get_chemical_symbols())):
    if f.get_chemical_symbols()[i] not in l:
        l.append(f.get_chemical_symbols()[i])
mix=str_permut2(l)
for a in l:
    mix.append(a+'-'+a)
text1=widgets.HTML(value="<h3>Structure multiplication:(MxNxP)</h3>")
w1=widgets.IntSlider(value=1,min=1,max=3,step=1,description='M',disabled=False,continuous_update=False,orientation='horizontal',readout=True)
w2=widgets.IntSlider(value=1,min=1,max=3,step=1,description='N',disabled=False,continuous_update=False,orientation='horizontal',readout=True)
w3=widgets.IntSlider(value=1,min=1,max=3,step=1,description='P',disabled=False,continuous_update=False,orientation='horizontal',readout=True)
text2=widgets.HTML(value="<h3>Bond properties:</h3>")
w=widgets.FloatText(value=-1.0,description='Max.distance:',disabled=False)
drop=widgets.SelectMultiple(options=mix,description='Bonds',disabled=False)
radius=widgets.FloatSlider(value=0.15,min=0,max=0.3,step=0.05,description='Radius',disabled=False,continuous_update=False,orientation='horizontal',readout=True)
text22=widgets.HTML(value="<p>-to select multiple bonds, hold the 'ctrl' button and select all the desired bonds.<br>-if we put a negative number in 'max.distance', bonds will be assigned by default options.</p>")
helpx= widgets.Accordion(children=[text22], titles=tuple(['Help']))
text3=widgets.HTML(value="<h3>Bond color:</h3>")
c1=widgets.FloatSlider(value=1,min=0,max=1.0,step=0.1,description='Red',disabled=False,continuous_update=False,orientation='horizontal',readout=True)
c2=widgets.FloatSlider(value=0.8,min=0,max=1.0,step=0.1,description='Green',disabled=False,continuous_update=False,orientation='horizontal',readout=True)
c3=widgets.FloatSlider(value=0,min=0,max=1.0,step=0.1,description='Blue',disabled=False,continuous_update=False,orientation='horizontal',readout=True)
#[text1,w1,w2,w3,text2,w,drop,radius,text3,c1,c2,c3]
child1=widgets.VBox([text1,w1,w2,w3])
child2=widgets.VBox([text2,w,drop,radius,helpx])
child3=widgets.VBox([text3,c1,c2,c3])
children=[child1,child2,child3]
tab = widgets.Tab()
tab.children = children
tab.titles = ['structure','bond','bond+']
def tup_conc(a,b):
    x=[]
    y=[]
    for i in a:
        x.append(i)
    for i in b:
        y.append(i)
    return(tuple(x+y))
def multiple(a,b,c,nx,d,radius,c1,c2,c3):   
    global ui
    global xview
    global p
    x=f.get_positions()
    n=a*b*c
    cell=f.get_cell()
    try:
        product=cartesian(a,b,c)
    except:
        return
    for k in range(n-1):
        y=f.get_positions()
        for i in range(y.shape[0]):
            for j in range(3):
                y[i][j]=y[i][j]+int(product[k][0])*cell[0][j]+int(product[k][1])*cell[1][j]+int(product[k][2])*cell[2][j]
        x=np.concatenate((x,y), axis=0)
    at=Atoms(symbols=f.get_chemical_symbols()*n,cell=cell,positions=x)
    if nx>=0:
        xview=view(at,viewer='ngl').children[0]
        xview.add_unitcell()
    else:
        xview=view_structure(at)
        xview.add_unitcell()
        xview.add_ball_and_stick()    
        xview.parameters = {"sele": "all","ballAndStick": {"multipleBond": "symmetric", "forceBonds": True}}
        xview.camera = 'orthographic'
    p=permut2(len(at.positions))
    if nx>=0:    
        for i in range(len(p)):
            a=at.get_chemical_symbols()[p[i][0]]
            b=at.get_chemical_symbols()[p[i][1]]
            if distance(at.positions[p[i][0]],at.positions[p[i][1]])<nx and str(a+'-'+b) in d:
                xview.shape.add_cylinder(at.positions[p[i][0]], at.positions[p[i][1]],[c1,c2,c3], radius)
    right_box=xview
    text_atom=widgets.HTML(value="<p><b>Number of atoms: </b>"+str(len(at.get_positions()))+"</p>")
    ui=widgets.HBox([tab, right_box])
    display(ui)
    display(text_atom)
out =interactive_output(multiple, {'a':w1, 'b':w2, 'c':w3,'nx':w,'d':drop,'radius':radius,'c1':c1,'c2':c2,'c3':c3})
#display(out)
def download_image_callback(change):
    xview.download_image(filename="nglwidget_output.png", factor=10, antialias=True)
def save_as(f,txt,value_inside):
    p=filedialog.askdirectory()
    path=str(p)
    inp=txt.get(1.0, "end-1c") 
    val=value_inside.get()
    save=inp+'.'+val
    if path=='' or inp=='':
        return
    else:
        path=path+'/'+save
        write(path,f)
b=widgets.Button(description="Download as FILE")
output = widgets.Output()
def window(b):
    with output:
        win=Tk()
        win.geometry('370x80')
        win.wm_overrideredirect(1)
        win.wm_geometry("+%d+%d" % (500, 300))
        txt=Text(win,height=1,width=20) 
        txt.place(x=50,y=30)
        Fact = "test"
        txt.insert(END, Fact)
        options_list=['cif','xyz','vasp']
        value_inside=StringVar(win)
        value_inside.set(options_list[0])
        question_menu=OptionMenu(win, value_inside, *options_list) 
        question_menu.place(x=215,y=25) 
        submit=Button(win, text = "save")
        submit['command']=partial(save_as,f,txt,value_inside)
        submit.place(x=285,y=27)
        exit=Button(win, text = "Exit",command = win.destroy)
        exit.place(x=320,y=27)
        win.mainloop()
b.on_click(window)
download_button = widgets.Button(description="Download as PNG")
text4=widgets.HTML(value="<h3>Download options:</h3>")
lay = widgets.Layout(display='flex',flex_flow='column',align_items='center',width='100%')
middle=widgets.VBox([text4,download_button,b],layout=lay)
display(out)
#display(widgets.VBox([middle],layout=lay))
tab.children=tup_conc(tab.children,tuple([middle]))
tab.titles=tup_conc(tab.titles[0:-1],tuple(['download']))
download_button.on_click(download_image_callback)
#tab.children=tup_conc(tuple([widgets.VBox([tab.children[0],text_atom])]),tab.children[1:])


#tup_conc(tab.titles[0:len(tab.titles)-1],tuple(['download']))